In [4]:
from IPython.core.display import display
import random
from keras.models import load_model
from keras.callbacks import EarlyStopping, ModelCheckpoint
# from IPython.display import Image
from keras.utils import plot_model
from keras.optimizers import Adam
from keras.layers import LSTM, Dense, Dropout, Embedding, Masking, Bidirectional
from keras.models import Sequential, load_model
import gc
import sklearn
import pickle
from sklearn.utils import shuffle
import re
from keras.preprocessing.text import Tokenizer
import numpy as np
import pandas as pd
from IPython.display import HTML

/var/folders/mh/2sqd7p2s0331dwc0z2rzgtl40000gn/T/ipykernel_44111/1320163270.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display


ModuleNotFoundError: No module named 'keras'

from google.colab import drive

In [64]:
data = pd.read_csv('./neural_network_patent_query.csv',
                   parse_dates=['patent_date'])
original_abstracts = list(data['patent_abstract'])

In [65]:
def format_patent(patent):
    """Add spaces around punctuation
    and remove references to images/citations."""

    # Add spaces around punctuation
    patent = re.sub(r'(?<=[^\s0-9])(?=[.,;?])', r' ', patent)

    # Remove references to figures
    patent = re.sub(r'\((\d+)\)', r'', patent)

    # Remove double spaces
    patent = re.sub(r'\s\s', ' ', patent)
    return patent

formatted = []

# Iterate through all the original abstracts and apply formatting<br>
for a in original_abstracts:<br>
    formatted.append(format_patent(a))

# Features and Labels<br>
# Note to self: new_texts and new_sequences are the texts and corresponding sequences<br>
# from only the abstracts that are 50 or more words (a sequence is vector of ints representing a piece of text)

def make_sequences(texts,<br>
                   training_length=50,<br>
                   lower=True,<br>
                   filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n'):<br>
#     
Turn a set of texts into sequences of integers
<br>
#     # Create the tokenizer object and train on texts<br>
#     tokenizer = Tokenizer(lower=lower, filters=filters)<br>
#     tokenizer.fit_on_texts(texts)<br>
#     # Create look-up dictionaries and reverse look-ups<br>
#     word_idx = tokenizer.word_index<br>
#     idx_word = tokenizer.index_word<br>
#     num_words = len(word_idx) + 1<br>
#     word_counts = tokenizer.word_counts<br>
#     print(f'There are {num_words} unique words.')<br>
#     # Convert text to sequences of integers<br>
#     sequences = tokenizer.texts_to_sequences(texts)<br>
#     # Limit to sequences with more than training length tokens<br>
#     seq_lengths = [len(x) for x in sequences]<br>
#     over_idx = [<br>
#         i for i, l in enumerate(seq_lengths) if l > (training_length + 20)<br>
#     ]<br>
#     new_texts = []<br>
#     new_sequences = []<br>
#     # Only keep sequences with more than training length tokens<br>
#     for i in over_idx:<br>
#         new_texts.append(texts[i])<br>
#         new_sequences.append(sequences[i])<br>
#     training_seq = []<br>
#     labels = []<br>
#     # Iterate through the sequences of tokens<br>
#     for seq in new_sequences:<br>
#         # Create multiple training examples from each sequence<br>
#         for i in range(training_length, len(seq)):<br>
#             # Extract the features and label<br>
#             extract = seq[i - training_length:i + 1]<br>
#             # Set the features and label<br>
#             training_seq.append(extract[:-1])<br>
#             labels.append(extract[-1])<br>
#     print(f'There are {len(training_seq)} training sequences.')<br>
#     # Return everything needed for setting up the model<br>
#     return word_idx, idx_word, num_words, word_counts, new_texts, new_sequences, training_seq, labels<br>
# TRAINING_LENGTH = 50<br>
# filters = '!"%;[\\]^_`{|}~\t\n'<br>
# word_idx, idx_word, num_words, word_counts, abstracts, sequences, features, labels = make_sequences(<br>
#     formatted, TRAINING_LENGTH, lower=False, filters=filters)<br>
# # Create training and validation sets. One-hot encoding labels<br>
# def create_train_valid(features,<br>
#                        labels,<br>
#                        num_words,<br>
#                        train_fraction=0.7):<br>
    
Create training and validation features and labels.


    # Randomly shuffle features and labels<br>
    features, labels = shuffle(features, labels, random_state=50)

    # Decide on number of samples for training<br>
    train_end = int(train_fraction * len(labels))

    train_features = np.array(features[:train_end])<br>
    valid_features = np.array(features[train_end:])

    train_labels = labels[:train_end]<br>
    valid_labels = labels[train_end:]

    # Convert to arrays<br>
    X_train, X_valid = np.array(train_features), np.array(valid_features)

    # Using int8 for memory savings<br>
    y_train = np.zeros((len(train_labels), num_words), dtype=np.int8)<br>
    y_valid = np.zeros((len(valid_labels), num_words), dtype=np.int8)

    # One hot encoding of labels<br>
    for example_index, word_index in enumerate(train_labels):<br>
        y_train[example_index, word_index] = 1

    for example_index, word_index in enumerate(valid_labels):<br>
        y_valid[example_index, word_index] = 1

    # Memory management: one-hot encoding the labels creates massive numpy arrays<br>
    # so I took care to delete the un-used objects from the workspace.<br>
    import gc<br>
    gc.enable()<br>
    del features, labels, train_features, valid_features, train_labels, valid_labels<br>
    gc.collect()

    return X_train, X_valid, y_train, y_valid

X_train, X_valid, y_train, y_valid = create_train_valid(<br>
    features, labels, num_words)

# Save the intermediate results as it is consuming too much memory!!

def save_intermediate_results(datap):<br>
    for i in data:<br>
        with open(f'./Datasets/{i}.pkl', 'wb') as f:<br>
            pickle.dump(globals()[i], f)

data = ['word_idx', 'idx_word', 'num_words', 'word_counts',<br>
        'abstracts', 'sequences', 'features', 'labels', 'X_valid', 'y_valid']<br>
save_intermediate_results(data)

with open('./Datasets/X_valid.pkl', 'rb') as f:<br>
    X_valid = pickle.load(f)<br>
    print(X_valid.shape)

# Clean up<br>
gc.enable()<br>
del (word_idx, idx_word, num_words, word_counts, abstracts,<br>
     sequences, features, labels, X_valid, y_valid)<br>
gc.collect()

# Build Model: After converting the words into embeddings, we pass them through<br>
# a single LSTM layer, then into a fully connected layer with relu activation<br>
# before the final output layer with a softmax activation

def make_word_level_model(num_words,<br>
                          lstm_cells=64,<br>
                          trainable=True,<br>
                          lstm_layers=1,<br>
                          bi_direc=False):<br>
#     
Make a word level recurrent neural network with option for pretrained embeddings<br>
       and varying numbers of LSTM cell layers.


    model = Sequential()

    model.add(<br>
        Embedding(<br>
            input_dim=num_words,<br>
            output_dim=100,<br>
            input_length=50,<br>
            trainable=True))

    # If want to add multiple LSTM layers<br>
    if lstm_layers > 1:<br>
        for i in range(lstm_layers - 1):<br>
            model.add(<br>
                LSTM(<br>
                    lstm_cells,<br>
                    return_sequences=True,<br>
                    dropout=0.1,<br>
                    recurrent_dropout=0.1))

    # Add final LSTM cell layer<br>
    if bi_direc:<br>
        model.add(<br>
            Bidirectional(<br>
                LSTM(<br>
                    lstm_cells,<br>
                    return_sequences=False,<br>
                    dropout=0.1,<br>
                    recurrent_dropout=0.1)))<br>
    else:<br>
        model.add(<br>
            LSTM(<br>
                lstm_cells,<br>
                return_sequences=False,<br>
                dropout=0.1,<br>
                recurrent_dropout=0.1))

    model.add(Dense(128, activation='relu'))

    # Dropout for regularization<br>
    model.add(Dropout(0.5))

    # Output layer<br>
    model.add(Dense(num_words, activation='softmax'))

    # Compile the model<br>
    model.compile(<br>
        optimizer='adam',<br>
        loss='categorical_crossentropy',<br>
        metrics=['accuracy'])

    return model

model = make_word_level_model(<br>
    num_words=16192,<br>
    lstm_cells=64,<br>
    trainable=True,<br>
    lstm_layers=1)<br>
model.summary()

plot_model(model, to_file=f'./Datasets/patent abstract.png', show_shapes=True)

# Train Model!<br>
# Early Stopping: Stop training when validation loss no longer decreases<br>
# Model Checkpoint: Save the best model on disk

# Patience: Number of epochs with no improvement after which training will be stopped

def make_callbacks(model_name, save=True):<br>
#     
Make list of callbacks for training
<br>
#     # callbacks = [EarlyStopping(monitor='val_loss', patience=5)]<br>
#     callbacks = []<br>
#     if save:<br>
#         callbacks.append(<br>
#             ModelCheckpoint(<br>
#                 f'./Datasets/patent_abstracts.h5',<br>
#                 save_best_only=True,<br>
#                 save_weights_only=False))<br>
#     return callbacks<br>
# callbacks = make_callbacks(model)<br>
# # Verbose is for showing output<br>
# # Batch_size defines the number of samples that will be propagated through the network<br>
# # Typically networks train faster with mini-batches. That's because we update the weights after each propagation.<br>
# # If we used all samples during propagation we would make only 1 update for the network's parameter.<br>
# with open('./Datasets/X_valid.pkl', 'rb') as f:<br>
#     X_valid = pickle.load(f)<br>
#     print(X_valid.shape)<br>
# with open('./Datasets/y_valid.pkl', 'rb') as f:<br>
#     y_valid = pickle.load(f)<br>
#     print(y_valid.shape)<br>
# history = model.fit(<br>
#     X_train,<br>
#     y_train,<br>
#     epochs=5,<br>
#     batch_size=128,<br>
#     verbose=1,<br>
#     callbacks=callbacks,<br>
#     validation_data=(X_valid, y_valid))<br>
# model.save('patents_abstracts')<br>
def load_values(data):<br>
    val = []<br>
    for i in data:<br>
        with open(f'./Datasets/{i}.pkl', 'rb') as f:<br>
            globals()[f'{i}'] = pickle.load(f)<br>
            print(type(globals()[f'{i}']))<br>
            val.append(globals()[f'{i}'])<br>
    return val<br>
X_valid, y_valid = load_values(['X_valid', 'y_valid'])<br>
X_valid.shape<br>
model = load_model("patents_abstracts")<br>
results = model.evaluate(X_valid, y_valid, batch_size=128)<br>
with open('./Datasets/idx_word.pkl', 'rb') as f:<br>
    idx_word = pickle.load(f)<br>
with open('./Datasets/sequences.pkl', 'rb') as f:<br>
    sequences = pickle.load(f)<br>
print(len(sequences))<br>
print(sequences[0])<br>
str_out = ""<br>
for i in sequences[0]:<br>
    str_out += idx_word.get(i) + " "<br>
def generate_output(model,<br>
                    sequences,<br>
                    training_length=75,<br>
                    new_words=50,<br>
                    diversity=1,<br>
                    return_output=True,<br>
                    n_gen=1):<br>
  
Generate `new_words` words of output from a trained model and format into HTML.


In [66]:
def load_values(data):
    val = []
    for i in data:

        with open(f'./Datasets/{i}.pkl', 'rb') as f:
            globals()[f'{i}'] = pickle.load(f)
            print(type(globals()[f'{i}']))
            val.append(globals()[f'{i}'])
    return val


X_valid, y_valid = load_values(['X_valid', 'y_valid'])
X_valid.shape


model = load_model("patents_abstracts")
results = model.evaluate(X_valid, y_valid, batch_size=128)

with open('./Datasets/idx_word.pkl', 'rb') as f:
    idx_word = pickle.load(f)

with open('./Datasets/sequences.pkl', 'rb') as f:
    sequences = pickle.load(f)

print(len(sequences))
print(sequences[0])
str_out = ""
for i in sequences[0]:
    str_out += idx_word.get(i) + " "


def generate_output(model,
                    sequences,
                    training_length=75,
                    new_words=50,
                    diversity=1,
                    return_output=True,
                    n_gen=1):
    """Generate `new_words` words of output from a trained model and format into HTML."""

    # Choose a random sequence
    seq = random.choice(sequences)

    # Choose a random starting point
    seed_idx = random.randint(0, len(seq) - training_length - 10)
    # Ending index for seed
    end_idx = seed_idx + training_length

    gen_list = []

    for n in range(n_gen):
        # Extract the seed sequence
        seed = seq[seed_idx:end_idx]
        original_sequence = [idx_word[i] for i in seed]
        generated = seed[:] + ['#']

        # Find the actual entire sequence
        actual = generated[:] + seq[end_idx:end_idx + new_words]

        # Keep adding new words
        for i in range(new_words):

            # Make a prediction from the seed
            preds = model.predict(np.array(seed).reshape(1, -1))[0].astype(
                np.float64)

            # Diversify
            preds = np.log(preds) / diversity
            exp_preds = np.exp(preds)

            # Softmax
            preds = exp_preds / sum(exp_preds)

            # Choose the next word
            probas = np.random.multinomial(1, preds, 1)[0]

            next_idx = np.argmax(probas)

            # New seed adds on old word
            seed = seed[1:] + [next_idx]
            generated.append(next_idx)

        # Showing generated and actual abstract
        n = []

        for i in generated:
            n.append(idx_word.get(i, ''))

        gen_list.append(n)

    a = []

    for i in actual:
        a.append(idx_word.get(i, ''))

    a = a[training_length:]

    gen_list = [
        gen[training_length:training_length + len(a)] for gen in gen_list
    ]

    if return_output:
        return original_sequence, gen_list, a



# display stuff


def header(text, color='black'):
    raw_html = f'<h1 style="color: {color};"><center>' + \
        str(text) + '</center></h1>'
    return raw_html


def box(text):
    raw_html = '<div style="border:1px inset black;padding:1em;font-size: 20px;">' + \
        str(text)+'</div>'
    return raw_html


def addContent(old_html, raw_html):
    old_html += raw_html
    return old_html


def remove_weird_grammar(str):
    return str.replace(' , ', ', ').replace(' . ', '. ')


def run_project():
    original_sequence, predicted_continuation, actual_continuation = generate_output(
    model, sequences, 50)
    seed_seq_string = ' '.join(i for i in original_sequence)
    seed_seq_string = remove_weird_grammar(seed_seq_string)
    pred_seq_string = ' '.join(i for i in predicted_continuation[0])
    pred_seq_string = remove_weird_grammar(pred_seq_string)
    actual_seq_string = ' '.join(i for i in actual_continuation)
    actual_seq_string = remove_weird_grammar(actual_seq_string)

    seed_html = ''
    seed_html = addContent(seed_html, header(
        'Seed Sequence', color='darkblue'))
    seed_html = addContent(seed_html,
                        box(seed_seq_string))

    pred_html = ''
    pred_html = addContent(pred_html, header(
        'Predicted Sequence', color='darkblue'))
    pred_html = addContent(pred_html,
                        box(pred_seq_string))

    actual_html = ''
    actual_html = addContent(actual_html, header(
        'Actual Sequence', color='darkblue'))
    actual_html = addContent(actual_html,
                        box(actual_seq_string))
    display(HTML(seed_html))
    display(HTML(pred_html))
    display(HTML(actual_html))


<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
747/747 [==============================] - 16s 21ms/step - loss: 5.2208 - accuracy: 0.2261
3255
[18, 6149, 3599, 3136, 2003, 12, 2, 426, 683, 1311, 28, 5, 154, 54, 27, 2, 9984, 20, 5, 2976, 102, 9985, 7, 193, 1425, 780, 12, 2, 9986, 1935, 1537, 4, 13, 6149, 3599, 213, 27, 2, 9987, 23, 224, 20, 118, 28, 9988, 2, 5259, 4208, 6, 1425, 7, 2, 1018, 337, 27, 2, 4685, 3600, 3, 4208, 6, 1425, 25, 84, 2, 9989, 225, 3, 3358, 4, 13, 6149, 3599, 2337, 7, 1, 1311, 90, 5, 1289, 283, 90, 1093, 1, 225, 3, 3358, 3, 1, 4685, 472, 5, 6, 203, 2, 519, 1312, 23, 3358, 22, 30, 7, 1, 1311, 25, 1578, 1, 1311, 7, 2, 169, 53, 35, 23, 1217, 7479, 173, 1, 224, 225, 3, 3358, 3, 1, 4685, 472, 4]


In [67]:
run_project()

1/1 [==============================] - 0s 12ms/step
